In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import torch.nn as nn
from torch.optim.lr_scheduler import ExponentialLR, StepLR
import torch.nn.functional as F

In [3]:
use_cuda = torch.cuda.is_available()

In [4]:
%cd /content/drive/MyDrive/hacklytics
!ls

/content/drive/MyDrive/hacklytics
 Drug_Repurposing	    Molecule_Generation.ipynb  'supercompf (1).csv'
 drug_repurposing_release   predicting_vina.ipynb


In [5]:
#!git clone https://github.com/virenvarma007/drug_repurposing_release
!pip install git+https://github.com/samoturk/mol2vec

  Cloning https://github.com/samoturk/mol2vec to /tmp/pip-req-build-pcmlvuoj
  Running command git clone --filter=blob:none --quiet https://github.com/samoturk/mol2vec /tmp/pip-req-build-pcmlvuoj
  Resolved https://github.com/samoturk/mol2vec to commit 850d944d5f48a58e26ed0264332b5741f72555aa
  Preparing metadata (setup.py) ... done


In [6]:
!pip install rdkit-pypi

In [7]:
!ls

 Drug_Repurposing	    Molecule_Generation.ipynb  'supercompf (1).csv'
 drug_repurposing_release   predicting_vina.ipynb


In [8]:
%cd ./drug_repurposing_release
%cd ./release
import numpy as np
from tqdm import tqdm, trange
import pickle
from rdkit import Chem, DataStructs
from stackRNN import StackAugmentedRNN
from data import GeneratorData
from utils import canonical_smiles

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns

/content/drive/MyDrive/hacklytics/drug_repurposing_release
/content/drive/MyDrive/hacklytics/drug_repurposing_release/release


Failed to find the pandas get_adjustment() function to patch
Failed to patch pandas - PandasTools will have limited functionality


In [9]:
gen_data_path = '../../Drug_Repurposing/ReLeaSE-master/ReLeaSE-master/data/chembl_22_clean_1576904_sorted_std_final.smi'

In [ ]:
tokens = ['<', '>', '#', '%', ')', '(', '+', '-', '/', '.', '1', '0', '3', '2', '5', '4', '7','6', '9', '8', '=', 'A', '@', 'C', 'B', 'F', 'I', 'H', 'O', 'N', 'P', 'S', '[', ']','\\', 'c', 'e', 'i', 'l', 'o', 'n', 'p', 's', 'r', '\n']
gen_data = GeneratorData(training_data_path=gen_data_path, delimiter='\t',
                         cols_to_read=[0], keep_header=True, tokens=tokens)

In [ ]:
def plot_hist(prediction, n_to_generate):
    print("Mean value of predictions:", prediction.mean())
    print("Proportion of valid SMILES:", len(prediction)/n_to_generate)
    ax = sns.kdeplot(prediction, shade=True)
    ax.set(xlabel='Predicted vina',
           title='Distribution of predicted vina for generated molecules')
    plt.show()
def estimate_and_update(generator, predictor, n_to_generate, **kwargs):
    generated = []
    pbar = tqdm(range(n_to_generate))
    for i in pbar:
        pbar.set_description("Generating molecules...")
        generated.append(generator.evaluate(gen_data, predict_len=120)[1:-1])

    sanitized = canonical_smiles(generated, sanitize=False, throw_warning=False)[:-1]
    unique_smiles = list(np.unique(sanitized))[1:]
    smiles, prediction, nan_smiles = predictor.predict(unique_smiles, get_features=get_fp)

    plot_hist(prediction, n_to_generate)

    return smiles, prediction

In [ ]:
hidden_size = 1500
stack_width = 1500
stack_depth = 200
layer_type = 'GRU'
lr = 0.001
optimizer_instance = torch.optim.Adadelta

my_generator = StackAugmentedRNN(input_size=gen_data.n_characters, hidden_size=hidden_size,
                                 output_size=gen_data.n_characters, layer_type=layer_type,
                                 n_layers=1, is_bidirectional=False, has_stack=True,
                                 stack_width=stack_width, stack_depth=stack_depth,
                                 use_cuda=use_cuda,
                                 optimizer_instance=optimizer_instance, lr=lr)

#model = torch.load('model/pytorch_resnet50.pth',map_location ='cpu')
#my_generator.torch.load('/content/drive/MyDrive/Drug_Repurposing/checkpoint_biggest_rnn',map_location ='tpu')
!pip install mordred
from data import PredictorData
from utils import get_desc, get_fp
from mordred import Calculator, descriptors
calc = Calculator(descriptors, ignore_3D=True)

In [ ]:
pred_data = PredictorData(path='/content/drug_repurposing_release/data/supercompcut.csv', get_features=get_fp)

In [ ]:
from predictor import VanillaQSAR
from sklearn.ensemble import RandomForestRegressor as RFR

model_instance = RFR
model_params = {'n_estimators': 500, 'n_jobs': 20}
my_predictor = VanillaQSAR(model_instance=model_instance,
                           model_params=model_params,
                           model_type='regressor')


In [ ]:
my_predictor.fit_model(pred_data, cv_split='random')

In [ ]:
def get_reward_min(smiles, predictor, invalid_reward=0.0, get_features=get_fp):
    mol, prop, nan_smiles = predictor.predict([smiles], get_features=get_features)
    if len(nan_smiles) == 1:
        return invalid_reward
    return np.exp(-2*prop[0]/3 + 3)

In [ ]:
from reinforcement import Reinforcement

my_generator_min = StackAugmentedRNN(input_size=gen_data.n_characters, hidden_size=hidden_size,
                                 output_size=gen_data.n_characters, layer_type=layer_type,
                                 n_layers=1, is_bidirectional=False, has_stack=True,
                                 stack_width=stack_width, stack_depth=stack_depth,
                                 use_cuda=use_cuda,
                                 optimizer_instance=optimizer_instance, lr=lr)
model_path = '/content/drive/MyDrive/Drug_Repurposing/checkpoint_biggest_rnn'
weights = torch.load(model_path, map_location ='cpu')
my_generator_min.load_state_dict(weights)

RL_min = Reinforcement(my_generator_min, my_predictor, get_reward_min)

n_to_generate = 200
n_policy_replay = 10
n_policy = 15
n_iterations = 50

def simple_moving_average(previous_values, new_value, ma_window_size=10):
    value_ma = np.sum(previous_values[-(ma_window_size-1):]) + new_value
    value_ma = value_ma/(len(previous_values[-(ma_window_size-1):]) + 1)
    return value_ma

def get_reward_min(smiles, predictor, invalid_reward=0.0, get_features=get_fp):
    mol, prop, nan_smiles = predictor.predict([smiles], get_features=get_features)
    if len(nan_smiles) == 1:
        return invalid_reward
    return np.exp(-2*prop[0]/3 + 3)


smiles_biased_min, prediction_biased_min = estimate_and_update(RL_min.generator,
                                                           my_predictor,
                                                           n_to_generate=10000)



In [ ]:
print(len(smiles_biased_min))
from numpy import save
save('/content/ReLeaSE/output2', smiles_biased_min)